In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/image-colorization/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5
/kaggle/input/image-colorization/ab/ab/ab3.npy
/kaggle/input/image-colorization/ab/ab/ab1.npy
/kaggle/input/image-colorization/ab/ab/ab2.npy
/kaggle/input/image-colorization/l/gray_scale.npy


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,InputLayer, Conv2DTranspose, Dropout, BatchNormalization, Input, Concatenate, Activation, concatenate ,RepeatVector ,Reshape ,UpSampling2D
from keras.initializers import RandomNormal
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.optimizers import Adam
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
import cv2
import PIL
from skimage import transform
from PIL import Image
import random
import h5py
import os
import plotly.graph_objects as go
import matplotlib.pyplot as plt


# ******> Loading the Data......**

In [8]:
images_gray = np.load("../input/image-colorization/l/gray_scale.npy")
images_ab1 = np.load("../input/image-colorization/ab/ab/ab1.npy")
images_ab2 = np.load("../input/image-colorization/ab/ab/ab2.npy")
images_ab3 = np.load("../input/image-colorization/ab/ab/ab3.npy")


In [9]:
#taking training data of 100
X_train = (images_gray[:100,:,:].astype('float')).reshape(100,224,224,1) #reshaping the input gray images
Y = (images_ab1[:100,:,:].astype('float'))


In [11]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.applications.xception import Xception,decode_predictions,preprocess_input
xception = Xception(weights='imagenet', include_top=True)
xception.graph = tf.get_default_graph()

91889664/91884032 [==============================] - 1s 0us/step


In [12]:
def create_Xception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with xception.graph.as_default():
        embed = xception.predict(grayscaled_rgb_resized)
    return embed

In [14]:
xcept_em = create_Xception_embedding(X_train)
embeddings = RepeatVector(28 * 28)(xcept_em)
layer_embeddings = Reshape(([28, 28, 1000]))(embeddings)

In [15]:
model_path = "./color_model.h5"
checkpoint = ModelCheckpoint(model_path,
                            monitor = "val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose = 1)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [16]:
#Encoder
embed_input = Input(shape=(28, 28, 1000))
encoder_input = Input(shape=(224, 224, 1,))
encoder_1 = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_2 = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_1)
encoder_3 = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_2)
encoder_4 = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_3)
encoder_5 = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_4)
encoder_6 = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_5)
encoder_7 = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_6)
encoder_output= Conv2D(256, (3,3), activation='relu', padding='same')(encoder_7)
#Fusion layer
fusion_output = concatenate([encoder_output, layer_embeddings], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output)
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output)
#Decoder layer
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
model = Model(inputs=[encoder_input,embed_input], outputs=decoder_output)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 112, 112, 64) 640         input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 112, 112, 128 73856       conv2d_12[0][0]                  
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 56, 56, 128)  147584      conv2d_13[0][0]                  
_______________________________________________________________________________________

In [ ]:
model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
history = model.fit(x=[X_train,layer_embeddings] ,y=Y,callbacks = [checkpoint,es], batch_size=5, epochs=1000,steps_per_epoch=1)

Train on 1 samples
Epoch 1/1000
1/1 [==============================] - 0s 2ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 18084.3008 - acc: 0.4482
Epoch 2/1000
1/1 [==============================] - 0s 2ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 20263.5039 - acc: 0.4483
Epoch 3/1000
1/1 [==============================] - 0s 2ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 17793.6621 - acc: 0.4483
Epoch 4/1000
1/1 [==============================] - 0s 2ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 18046.7070 - acc: 0.4483
Epoch 5/1000
1/1 [==============================] - 0s 2ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 18044.9395 - acc: 0.4476
Epoch 6/1000
1/1 [==============================] - 0s 2ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 17939.6055 - acc: 0.4476
Epoch 7/1000
1/1 [==============================] - 0s 2ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 17403.5156 - acc: 0.4477
Epoch 8/1000
1/1 [==============================] - 0s 2m

1/1 [==============================] - 0s 5ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 3399.3770 - acc: 0.5432
Epoch 63/1000
1/1 [==============================] - 0s 5ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 3568.5889 - acc: 0.5445
Epoch 64/1000
1/1 [==============================] - 0s 2ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 3599.2473 - acc: 0.5454
Epoch 65/1000
1/1 [==============================] - 0s 2ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 3310.3616 - acc: 0.5454
Epoch 66/1000
1/1 [==============================] - 0s 2ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 3443.4282 - acc: 0.5456
Epoch 67/1000
1/1 [==============================] - 0s 2ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 3373.4387 - acc: 0.5469
Epoch 68/1000
1/1 [==============================] - 0s 2ms/step - batch: 0.0000e+00 - size: 1.0000 - loss: 3202.0242 - acc: 0.5493
Epoch 69/1000
1/1 [==============================] - 0s 2ms/step - batch: 0.0000e+00 - siz

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()


In [ ]:
output = model.predict([X_train,layer_embeddings],steps=3)


In [ ]:
def get_LAB(image_l, image_ab  ):
       
    image_l = image_l.reshape((224, 224, 1))
    image_lab = np.concatenate((image_l, image_ab), axis=2)
    image_lab = image_lab.astype("uint8")
 
    image_rgb = cv2.cvtColor(image_lab, cv2.COLOR_LAB2RGB)
    image_rgb = Image.fromarray(image_rgb)
    return image_rgb
def get_LAB1(image_l  ):
    image_ab =  np.ones((224,224,2))*128
    image_l = image_l.reshape((224, 224, 1))
    image_lab = np.concatenate((image_l, image_ab), axis=2)
    image_lab = image_lab.astype("uint8")
 
    image_rgb = cv2.cvtColor(image_lab, cv2.COLOR_LAB2RGB)
    image_rgb = Image.fromarray(image_rgb)
    return image_rgb

In [ ]:
for i in range(50):
    pred = get_LAB(X_train[i],output[i])
    real = get_LAB(X_train[i],Y[i])
    original = get_LAB1(X_train[i])
    f, axarr = plt.subplots(1,3)
    axarr[0].title.set_text('Black and white')  
    axarr[1].title.set_text('Prediction')  
    axarr[2].title.set_text('original')  
    axarr[0].imshow(original)
    axarr[1].imshow(pred)
    axarr[2].imshow(real)
    